In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import sys
import os
import collections
import pickle
from scipy.integrate import quad
from matplotlib import pyplot as plt
from scipy.special import logit, expit
from scipy.stats import norm
import math

In [ ]:
def plot(ax, xs, ys, title=None, xlabel=None, ylabel=None):
    ax.plot(xs, ys)
    if title:
        ax.set_title(title)
    if xlabel:
        ax.set_xlabel(xlabel)
    if ylabel:
        ax.set_ylabel(ylabel)
    if title:
        ax.title.set_text(title)

## Visualize Probability Density Functions

In [ ]:
def f(u, v, std=.1, dist='normal'):
    if dist == 'normal':
        return norm.pdf(u, v, std) * (norm.cdf(1, v, std) - norm.cdf(0, v, std)) ** -1
    elif dist == 'lipschitz':
        if v - std < u and u < v + std:
            return 1/(min(1, v + std) - max(0, v - std))
        else:
            return 0
    elif dist == 'square':
        return (1-(u-v)**2) * (quad(lambda u: (1-(u-v)**2), 0, 1)[0]) ** -1
        
xs = np.linspace(0, 1, 50)
vs = [0, .25, .5]
full_ys = [[] for _ in range(len(vs))]
for i, v in enumerate(vs):
    full_ys[i] = [f(x, v) for x in xs]

fig, axs = plt.subplots(1, 3, figsize=(15,5))
for i, v in enumerate(vs):
    plot(axs[i], xs, full_ys[i], title='v={}'.format(v), xlabel='u', ylabel='pdf_e(v,u)')
plt.subplots_adjust(wspace=0.17, hspace=0.1)

In [ ]:
# check that f is a valid PDF - these should all be approximately equal to 1
zero_to_one = True
for v in [0, .2, .5, .75, 1]:
    if zero_to_one:
        print(quad(lambda u: f(u,v), 0, 1))
    else:
        print(quad(lambda u: f(u,v), -20, 20))

## Compute exact preimage sizes with nested integrals

In [ ]:
high = 1 # max support of the pdf

def pdf(u, v, dist='normal', std=.5):
    # return the value of the pdf of nbhd(v) at u
    if dist == 'uniform':
        # uniform distribution on [0,1]
        return 1
    elif dist == 'lipschitz':
        # uniform on [v-std, v+std]
        if v - std <= u and u <= v + std:
            return 1/(min(1, v+std)-max(0, v-std))
        else: 
            return 0
    elif dist == 'normal':
        # normal dist with mean=v, std=std, scaled to be in [0,1]
        return norm.pdf(u, v, std) * (norm.cdf(1, v, std) - norm.cdf(0, v, std)) ** -1

def e1(v, s, std=.1):
    # return the expected size of the preimage of v
    return s * quad(lambda u: pdf(u,v,std=std), v, high)[0] \
            * (quad(lambda w: pdf(w,v,std=std), v, high)[0]) ** (s-1)

def ex(v, s, std=.1, n=1):
    # return the expected size of the n'th preimage of v
    if n == 1:
        return e1(v, s, std=std)
    return e1(v, s, std) * (quad(lambda u: pdf(u,v, std=std), v, high)[0]) ** (-1) \
                    * quad(lambda u: pdf(u,v, std=std) * ex(u, s, std, n-1), v, high)[0]

def full_preimage(v, s, std=.1, n=5):
    # return the expected size of the full preimage up to n
    if not n:
        return 0
    return ex(v, s, std, n) + full_preimage(v, s, std, n-1)


## compute preimage sizes by running simulations
 - the methods in the previous cell can become slow after 3 or 4 nested integrals
 - the methods in the next cell give the same results as long as we run enough trials
 - specifically, e1, ex, and full_preimage will give the same output as s1, sx, and sample_preimage (averaged over trials)


In [ ]:
def sample(v, std=.3, dist='normal'):
    # sample a random point from the nbhd of v
    if dist == 'uniform':
        return np.random.rand()
    elif dist == 'lipschitz':
        return np.random.uniform(max(0, v-std), min(1, v+std))
    elif dist == 'normal':
        # rejection sampling
        u = np.random.rand()
        y = np.random.rand() * pdf(v, v, dist='normal', std=std)
        if y < pdf(u, v, dist='normal', std=std):
            return u
        else:
            return sample(v, std=std, dist='lipschitz')
    
def get_nbhd(v, s, std=.3):
    # sample a nbhd for v
    return [sample(v, std) for _ in range(s)]

def check_if_chosen(v, u, s, std=.3):
    # check if f(u)=v by sampling a nbhd for u
    return (v < u and v < min(get_nbhd(u, s-1, std)))

def check_if_local_min(v, s, std=.3):
    return all([v < u for u in get_nbhd(v, s, std)])

def s1(v, s, std=.3):
    # return the size of the preimage for v
    return sum([check_if_chosen(v, u, s, std) for u in get_nbhd(v, s, std)])

def sx(v, s, std=.3, n=1):
    # return the size of the n'th preimage of v
    if n == 1:
        return s1(v, s, std)
    return sum([sx(u, s, std, n-1) for u in get_nbhd(v, s, std) 
                if check_if_chosen(v, u, s, std)])

def sample_preimage(v, s, std=.3, n=1):
    # return the size of the full preimage up to n 
    if not n:
        return 0
    return sx(v, s, std, n) + sample_preimage(v, s, std, n-1)


In [ ]:
# check that sample() is working correctly
ys = []
for _ in range(1000):
    ys.append(sample(v=.25, std=.3, dist='normal'))
plt.hist(ys, bins=20)
print()


## Fit the global distribution of the datasets in NAS-Bench-201
- get the loss files from https://drive.google.com/drive/folders/1XdXLrwCTWPPZvXrvCeJ9r0FboMyNLOhS?usp=sharing
- cifar10_losses.pkl, cifar100_losses.pkl. ImageNet16-120_losses.pkl
    - each file is a size 15625 array of all validation losses in the dataset

In [ ]:
datasets = ['cifar10', 'cifar100', 'ImageNet16-120']
xlimits = [(8,20), (25,55), (53,82)]
n = 15625

global_stds = [.18, .1, .22]
index = 2

dataset = datasets[index]
xlimit = xlimits[index]
length = xlimit[1] - xlimit[0]

losses, _ = pickle.load(open('{}_losses.pkl'.format(dataset), 'rb'))
pruned_losses = [(l-xlimit[0]+1e-4)/length for l in losses if l < xlimit[1]]
weights = np.ones_like(pruned_losses) / len(pruned_losses) * (xlimit[1]-xlimit[0])
print('fraction unpruned', len(pruned_losses)/n)
plt.hist(pruned_losses, bins=25, density=1)

xs = np.linspace(0, 1, 100)
ys = [f(x, .25, std=global_stds[index], dist='normal') for x in xs]
plt.plot(xs, ys, color='red', linewidth=2)

plt.xlabel('validation loss')
plt.ylabel('percent of architectures')
plt.title('Histogram of validation losses for {}'.format(dataset))
print()

## Compute the probability that arch will converge close to opt 
### Theorem 5.1

In [ ]:
def generate_vs(size=50,
                threshold=.1):
    # generate a set of v's
    vs = [*np.linspace(0, threshold, size, endpoint=False), 
      *np.linspace(threshold, 1-threshold, size, endpoint=False),
      *np.linspace(1-threshold, 1, size, endpoint=True)]
    return vs

def compute_local_min_probs(vs,
                           s=24,
                           std=.3,
                           trials=1000,
                           save_to_file=False):
    # compute the prob v is a local minima
    local_min_probs = []
    for i, v in enumerate(vs):
        if i % (len(vs) / 10) == 0:
            print(i, 'of', len(vs))
        prob = 0
        for _ in range(trials):
            prob += check_if_local_min(v, s=s, std=std)
        prob /= trials
        local_min_probs.append((v, prob))
    
    if save_to_file:
        pickle.dump(local_min_probs, open('local_min_probs_{}.pkl'\
                                          .format(std.split('.')[-1]), 'wb'))
    return local_min_probs
        
def compute_preimage_sizes(vs,
                          s=24,
                          std=.3,
                          trials=10,
                          n=3,
                          save_to_file=False):
    # compute the size of preimages
    preimage_sizes = []
    for i, v in enumerate(vs):
        if i == len(vs)//100:
            print(i, 'of', len(vs))
        if i % (len(vs) / 10) == 0:
            print(i, 'of', len(vs))
        counts = 0
        for _ in range(trials):
            counts += sample_preimage(v, s=s, std=std, n=n)
        counts /= trials
        preimage_sizes.append((v, counts))
    if save_to_file:
        pickle.dump(preimage_sizes, open('preimage_sizes_{}.pkl'\
                                         .format(std.split('.')[-1]), 'wb'))
    return preimage_sizes
        
def compute_epsilon_probs(vs,
                         local_min_probs,
                         preimage_sizes,
                         s=24,
                         global_std=.3,
                         global_shift=.25,
                         trials=10000,
                         save_to_file=False):
    # now compute the full epsilon probabilities
    
    contribs = []
    for t in range(trials):
        if t % (trials / 10) == 0:
            print(t/trials)
        v = sample(v=global_shift, std=global_std, dist='normal')
        index = np.argmin([abs(v - u) for u in vs])
        contribs.append((v, local_min_probs[index][1] * preimage_sizes[index][1]))
        
    return contribs

In [ ]:
# check everything is working
vs = generate_vs(size=20, threshold=.1)
local_min_probs = compute_local_min_probs(vs, std=.3, trials=50)

print('finished local min probs')
plt.plot(vs, [y[1] for y in local_min_probs])
plt.xscale('log')
plt.show()

preimage_sizes = compute_preimage_sizes(vs, std=.3, trials=1, n=2)

print('finished preimage sizes')
plt.plot(vs, [y[1] for y in preimage_sizes])
plt.xscale('log')
plt.show()

contribs = compute_epsilon_probs(vs, 
                                 local_min_probs, 
                                 preimage_sizes,
                                 trials=1000,
                                 global_std=.5,
                                 global_shift=.25)
print('finished contribs')

total = sum([pair[1] for pair in contribs])
for eps in [.005, .01, .05, .1]:
    eps_total = sum([pair[1] for pair in contribs if pair[0] < eps])
    print('{} fraction: {}'.format(eps, eps_total / total))
    
xs = np.geomspace(.01, 1, 100)
ys = [sum([pair[1] for pair in contribs if pair[0] < x]) for x in xs]
ys = np.array(ys) / total
plt.plot(xs, ys)
plt.xscale('log')

## Compare Theorem 5.1 and Lemma 5.2 to real data
- first, run using our precomputed data files from https://drive.google.com/drive/folders/1XdXLrwCTWPPZvXrvCeJ9r0FboMyNLOhS?usp=sharing


In [ ]:
# compare real vs. simulated data using our precomputed data

datasets = ['cifar10', 'cifar100', 'Random', 'ImageNet16-120', ]
names = ['CIFAR-10', 'CIFAR-100', 'Unif. Random', 'ImageNet16-120']
snames = ['Thm 5.1 sim. - CIFAR-10', 'Thm 5.1 sim. - CIFAR-100', 'Thm 5.1 sim. - Unif. Random', 'Thm 5.1 sim. - ImageNet16-120']
colors = ['#00A6FF', '#FFB900', '#FF2D00', '#5DFF00']
scolors = ['#007FC2', '#C89100', '#B72100', '#2ED800']

xs = np.geomspace(.003, .1, 100)

fig, axes = plt.subplots()

# compute simulation from Lemma 5.2
def lemma_5_2(eps, s=24, n=30):
    return 1/(s+1) * (1- (1-eps)**(s+1)) \
        + (1-(1-eps)**(2*s+1)) * s/(s+1) \
        + (1-(1-eps)**s) * sum([(1-eps)**(i*s+1) * np.prod([s/(j*s+1) \
                                for j in range(1, i)]) for i in range(3, n)])

for i, dataset in enumerate(datasets):
    # plot the real dataset
    percents = pickle.load(open('{}_percents.pkl'.format(datasets[i]), 'rb'))
    ps = [sum([1 for percent in percents if (percent < x and percent < 1)]) for x in xs]
    ps = np.array(ps) / ps[-1]
    plt.plot(xs, ps, label=names[i], color=colors[i])

    # plot the synthetic dataset
    if dataset != 'Random':
        contribs = pickle.load(open('synth_{}.pkl'.format(datasets[i]), 'rb'))
        cs = np.array([sum([pair[1] for pair in contribs if pair[0] < x]) for x in xs])
        cs /= cs[-1]
        plt.plot(xs, cs, label='synth_{}'.format(datasets[i]), linewidth=2.5, linestyle='dashed', color=scolors[i])
    else:
        fs = [lemma_5_2(x) for x in xs]
        plt.plot(xs, fs, label='synth_Random', linewidth=2.5, linestyle='dashed', color=scolors[i])

lines = axes.get_lines()
legend1 = plt.legend([lines[w] for w in [4, 6, 0, 2]], [names[n] for n in [2,3,0,1]], loc='upper left')
legend2 = plt.legend([lines[w] for w in [5, 7, 1, 3]], [snames[n] for n in [2,3,0,1]], loc='lower right')
axes.add_artist(legend1)
axes.add_artist(legend2)
plt.xscale('log')
#plt.legend(loc='best')
plt.xlabel('distance to global minimum')
plt.ylabel('percent of architectures')
print()

In [ ]:
# run simulations from scratch (can be compute intensive)

global_params = [(.25, .18), (.25, .1), (.25, .22)]
local_params = [350, 350, 350]
datasets = ['cifar10', 'cifar100', 'ImageNet16-120', 'random']

trial_vars = [500, 5, 3, 500000]
xs = np.geomspace(.003, .1, 100)
    
# plot the dataset
for i, dataset in enumerate(datasets):
    percents = pickle.load(open('{}_percents.pkl'.format(datasets[i]), 'rb'))
    ps = [sum([1 for percent in percents if (percent < x and percent < 1)]) for x in xs]
    ps = np.array(ps) / ps[-1]
    plt.plot(xs, ps, label=dataset)
    
# plot new arrays + local contrib function
vs = generate_vs(size=50, threshold=.1)

for i in range(3):

    local_min_probs = compute_local_min_probs(vs, std=n, trials=trial_vars[0])
    preimage_sizes = compute_preimage_sizes(vs, std=n, trials=trial_vars[1], n=trial_vars[2])
    
    contribs = compute_epsilon_probs(vs, 
                                     local_min_probs, 
                                     preimage_sizes,
                                     trials=trial_vars[3],
                                     global_std=global_params[i][1],
                                     global_shift=global_params[i][0])

    cs = np.array([sum([pair[1] for pair in contribs if pair[0] < x]) for x in xs])
    cs /= cs[-1]
    plt.plot(xs, cs, label='synth_{}'.format(datasets[i]))

# add random nasbench-201 data

fs = [lemma_5_2(x) for x in xs]
plt.plot(xs, fs, label='synth_random', linewidth=2.5, alpha=0.7)
    
plt.xscale('log')
plt.legend(loc='best')
plt.xlabel('distance to global minimum')
plt.ylabel('percent of architectures')
print()

## Simulate preimage sizes for nas-bench-201 random data
- get random_preimage_sizes.pkl from https://drive.google.com/drive/folders/1XdXLrwCTWPPZvXrvCeJ9r0FboMyNLOhS?usp=sharing

In [ ]:
# plot the preimage sizes of all the local minima

preimage_sizes = pickle.load(open('random_preimage_sizes.pkl', 'rb'))

xs = [m[0] for m in preimage_sizes if m[0] < .2] # val losses of local minima
ys = [m[1] for m in preimage_sizes if m[0] < .2] # preimage sizes of local minima

# compute the average over a sliding window
window=100
avg_xs = [np.mean([xs[i:i+window]]) for i in range(len(xs)-1)]
avg_ys = [np.mean([ys[i:i+window]]) for i in range(len(ys)-1)]


def exact_bound(v, s=24, n=20):
    # this is Equation A.1 from Lemma A.2 in our paper
    return 1 + s * sum([(1-v)**(s*i) * np.prod([s/(j*s+1) for j in range(1, i)]) for i in range(1, n)])

thresh = .05
x_max = .2
size = 10000
vs= [*np.linspace(0, thresh, size, endpoint=False), 
      *np.linspace(thresh, x_max, size, endpoint=True)]

plt.plot(vs, [exact_bound(x) for x in vs], label='Lemma 5.2', linewidth=3, color='green', alpha=0.8)
plt.scatter(avg_xs, avg_ys, s=8.8, alpha=0.8, label='NASBench-201 with losses in U([0,1])', color='red')
plt.legend(loc='best')
plt.xlabel('Validation loss')
plt.ylabel('Size of full preimage')
plt.show()
